In [ ]:
# %pip install pymssql

In [57]:
import pymssql
import pandas as pd
import redis
import time
import datetime

In [58]:
REDIS_IP = '10.2.5.212'

In [59]:
def ms_sql_con():
    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()

    return pymssql.connect(
            server = sql_server,
            user = sql_login,
            password = sql_pass,
            database = sql_name,
            autocommit=True
        )

In [74]:
def read_sql(query):
    return pd.read_sql(query, con=ms_sql_con(), parse_dates=None)

In [109]:
def concatenate_linkedid_side(side, record_date, linkedid):
    query = "SELECT text from transcribations where "
    query += " side="+str(side)+" and "
    query += " record_date = '"+str(record_date)+"' and "
    query += " linkedid = '"+str(linkedid)+"';"
    text_df = read_sql(query)
    phrases_count = len(text_df)
    text_full = ', '.join([row.text for _id, row in text_df.iterrows()])
    return text_full, phrases_count

In [86]:
def summarize(text, phrases_count):
    if phrases_count<2 or len(text)<255:
        return text
    subscriber = redis.StrictRedis(host=REDIS_IP)
    publisher = redis.StrictRedis(host=REDIS_IP) 
    pub = publisher.pubsub()
    sub = subscriber.pubsub()
    sub.subscribe('summarus_client')
    # send
    publisher.publish("summarus_server", text)
    # receive
    while True:
        message = sub.get_message()
        if message and message['type']!='subscribe':
            return message['data'].decode("utf-8")
        time.sleep(1)

In [77]:
def sum_to_sql(linkedid, recor_date, side, text, phrases_count, text_length):
    current_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    query = "insert into summarization(linkedid, record_date, sum_date, side, text, phrases_count, text_length) "
    query += " values("
    query += "'"+str(linkedid)+"',"
    query += "'"+str(recor_date)+"',"
    query += "'"+str(current_date)+"',"
    query += str(side)+","
    query += "'"+str(text)+"',"
    query += "'"+str(phrases_count)+"',"
    query += "'"+str(text_length)+"'"
    query += ");"
    #df = read_sql(query)
    
    conn = ms_sql_con()  
    cursor = conn.cursor()
    cursor.execute(query)
    # conn.commit()

In [ ]:
# queue
query = "SELECT column_name FROM information_schema.columns WHERE table_name='queue';"
queue = read_sql(query)
queue

In [ ]:
# transcribations
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
queue = read_sql(query)
queue

### queue linkedid list

In [ ]:
# queue
query = "SELECT distinct record_date, linkedid from queue;"
queue = read_sql(query)
len(queue)

### transcribations linkedid list

In [ ]:
# queue
query = "SELECT distinct top 10 record_date, linkedid from transcribations;"
df = read_sql(query)
df

In [ ]:
query = "SELECT distinct top 100 trans.record_date, trans.linkedid"
query += " from transcribations as trans"
query += " left join queue as queue"
query += " on trans.record_date=queue.record_date"
query += " and trans.linkedid=queue.linkedid"
query += " where "
query += " trans.record_date > '2021-06-04 00:00:00' and"
query += " isnull(queue.linkedid,'x')='x'"
query += " order by record_date desc;"
%time df = read_sql(query)
len(df)

# start

In [123]:
# obtain datetime limits
query = "select min(record_date) from queue;"
df = read_sql(query)
queue_first_record = str(df.iloc()[0][0])
query = "select max(record_date) from summarization;"
df = read_sql(query)
summarization_first_record = str(df.iloc()[0][0])

In [124]:
query = "SELECT distinct top 1000 record_date, linkedid"
query += " from transcribations"
query += " where "
query += " record_date < '"+queue_first_record+"' and"
#query += " record_date >= '"+summarization_first_record+"'"
#query += " not linkedid in (select distinct linkedid from queue) and"
query += " not linkedid in (select distinct linkedid from summarization)"
query += " order by record_date desc;"
%time df = read_sql(query)
len(df)

CPU times: user 4.79 ms, sys: 710 µs, total: 5.5 ms
Wall time: 46.5 s


6

In [125]:
#text_full

In [126]:
for _id, row in df.iterrows():
    
    for side in range(2):
        text_full, phrases_count = concatenate_linkedid_side(side, row.record_date, row.linkedid)
        text_short = summarize(text_full, phrases_count)
        sum_to_sql(row.linkedid, row.record_date, side, text_short, phrases_count, len(text_full))

In [128]:
len('Российский мастер по ремонту стиральных машин, с кем можно переговорить, пожаловался на то, что белье не попадает в барабан, барабан не крутится, не вращается, он перетащил машинку с одного места на второе, а она почему-то перестала стирать, да, да нет москва, а я давно как ты взял эту самую визитку у этого почтовом ящике и вот по ней и звоню айсберг вот визитка, да конечно номер триста шестнадцать серия, спасибо спасибо, да пожалуй, да.')

441

In [127]:
# select
query = "SELECT * from summarization;"
df = read_sql(query)
for _id, row in df.iterrows():
    #df.text.iloc()[3]
    print(row.record_date, row.linkedid, row.text)

2021-06-04 12:18:04 1622798282.288293 Westphalt Contracting Corporation (WPC) выступил в защиту компании айсберг, которая является одним из крупнейших производителей ламинат в мире.
2021-06-04 12:18:04 1622798282.288293 На днях у меня сломал руку мастер, который меня заменил ламинат в квартире.
2021-06-04 12:17:59 1622798278.236092 
2021-06-04 12:17:59 1622798278.236092 здравствуйте вас приветствует единая служба сервиса сожалению сейчас все операторы заняты ваш звонок очень важен для нас пожалуйста дождитесь ответа оператора
2021-06-04 12:17:56 1622798275.236091 
2021-06-04 12:17:56 1622798275.236091 
2021-06-04 12:17:52 1622798271.236090 
2021-06-04 12:17:52 1622798271.236090 здравствуйте вас приветствует единая служба сервиса сожалению сейчас все операторы заняты ваш звонок очень важен для нас пожалуйста дождитесь ответа оператора
2021-06-04 12:20:07 1622798405.236172 Российский мастер по ремонту стиральных машин, с кем можно переговорить, пожаловался на то, что белье не попадает в 

In [73]:
row

linkedid                                         1622796230.235140
record_date                                    2021-06-04 11:43:52
sum_date                                       2021-06-04 14:35:00
side                                                          True
load_msk                                                      None
load_spb                                                      None
load_reg                                                      None
text             Westwood и его партнёры уже не первый год пров...
phrases_count                                                    2
text_length                                                    196
Name: 3, dtype: object

query = "SELECT column_name FROM information_schema.columns WHERE table_name='summarization';"
queue = read_sql(query)
queue

In [122]:
# select
query = "SELECT side, text from transcribations where "
query += " record_date = '2021-06-04 12:18:08' and linkedid = '1622798285.288298';"
df = read_sql(query)
', '.join([row.text for _id, row in df.iterrows()])

'здравствуй ещё раз от мастера вас беспокоит, выезжаю к вам через час примерно подъеду, скажите пожалуйста номер подъезда этаж квартиры код домофона, второй, шестьдесят сколько, номер квартира, хорошо выиграю, и, алло, так, отлично вас ждут с нетерпением, второй подъезд, восьмой этаж, домофон шестьдесят пять, пять шестьдесят пять, да сэр, бальзамо'

In [33]:
#df.text.iloc()[3]

'Germany. Москва, улица матроса железняка дом тридцать шесть квартира шестьдесят семь. подъезд четыре этаж два код домофона четыре пятнадцать. двенадцати до четырнадцати. досками.'

# DELETE

In [113]:
# delete
query = "DELETE from summarization;"
conn = ms_sql_con()  
cursor = conn.cursor()
cursor.execute(query)